In [1]:
from ase.build import bulk
import matplotlib.pyplot as plt
import numpy as np

In [2]:
import quantum_espresso
from pyiron_base import Project

# Workflow

## Structure Optimization

In [3]:
pseudopotentials = {"Al": "Al.pbe-n-kjpaw_psl.1.0.0.UPF"}

In [4]:
pr = Project("test")
pr.remove_jobs(recursive=True, silently=True)

  0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
structure = bulk('Al', a=4.15, cubic=True)

In [6]:
job_qe_minimize = pr.create.job.QEMinimize(job_name="qe_relax")

In [7]:
job_qe_minimize.input

DataContainer({'structure': None, 'pseudopotentials': {'Al': 'Al.pbe-n-kjpaw_psl.1.0.0.UPF'}, 'kpts': (3, 3, 3)})

In [8]:
job_qe_minimize.input.structure = structure

In [ ]:
job_qe_minimize.run()

The job qe_relax was saved and received the ID: 1


In [ ]:
structure_opt = job_qe_minimize.output.structure
structure_opt

## Energy Volume Curve

In [ ]:
energy_lst, volume_lst = [], []
for i, strain in enumerate(np.linspace(0.9, 1.1, 5)):
    structure_strain = structure_opt.copy()
    structure_strain = structure.copy()
    structure_strain.set_cell(structure_strain.cell * strain**(1/3), scale_atoms=True)
    job_strain = pr.create.job.QEStatic(job_name="job_strain_" + str(i))
    job_strain.input.structure = structure_strain
    job_strain.run()
    energy_lst.append(job_strain.output.energy)
    volume_lst.append(job_strain.output.volume)

# Result

In [ ]:
plt.plot(volume_lst, energy_lst)
plt.xlabel("Volume")
plt.ylabel("Energy")

# Summary

In [ ]:
pr.job_table()